In [50]:
import torch
from torch import nn 
import numpy as np
import torch.optim as optim
class TrisModel(torch.nn.Module):
    def __init__(self):
        super(TrisModel, self).__init__()
        self.lin1 = torch.nn.Linear(9,100)
        self.activation1 = torch.nn.LeakyReLU()
        
        self.lin2 = torch.nn.Linear(100,500)
        self.activation2 = torch.nn.LeakyReLU()
        
        self.lin3 = torch.nn.Linear(500,9)
        #self.activation3 = torch.nn.LeakyReLU()
        #self.softmax = torch.nn.Softmax()
    def forward(self, x):
        #print(x.shape)
        x = self.lin1(x)
        x = self.activation1(x)
        
        x = self.lin2(x)
        x = self.activation2(x)
        
        x = self.lin3(x)
        #x = self.softmax(x)
        return x


In [51]:
model = TrisModel()
with torch.no_grad():
    out= model(torch.ones(9))
print(out)

tensor([-0.3266,  0.2102,  0.0772,  0.0033, -0.1859,  0.0756,  0.1574,  0.0276,
         0.2222])


In [ ]:
def double_q_learning_loss(q1, q2, states, actions, rewards, next_states, dones, gamma=0.99):
    # Get Q-values for current state-action pairs from Q1
    q_values = q1(states)  # shape: [batch_size, num_actions]
    q_value = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)  # shape: [batch_size]

    # Get next state action values from Q1
    next_q_values = q1(next_states)  # shape: [batch_size, num_actions]
    next_actions = torch.argmax(next_q_values, dim=1)  # shape: [batch_size]

    # Evaluate those actions using q2 (Q2)
    next_q_target_values = q2(next_states)
    next_q_value = next_q_target_values.gather(1, next_actions.unsqueeze(1)).squeeze(1)

    # Compute target
    expected_q_value = rewards + gamma * next_q_value * (1 - dones)

    # Compute loss
    loss = F.mse_loss(q_value, expected_q_value.detach())

    return loss

In [70]:
import copy
import torch.nn.functional as F
def train_identity(model, epochs=1000, batch_size=32, lr=0.001, device='cpu'):
    EPOCHS_BEHIND = 10
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    with torch.no_grad():
        out= model(torch.tensor([1.,23,2,1,4,5,6,7,3],))
    print(out)
    
    q2_model = copy.deepcopy(model)
    next_q2_model = copy.deepcopy(model)

    for epoch in range(epochs):
        if epoch % EPOCHS_BEHIND == 0:
            # The q2_model is EPOCHS_BEHIND epochs behind model
            q2_model = copy.deepcopy(next_q2_model)
            next_q2_model = copy.deepcopy(model)
        # Generate random input data
        inputs = torch.rand(batch_size, 9).to(device)
        targets = inputs.clone()  # Identity target

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.6f}')
            
            
    return q2_model

# Example usage
model = TrisModel()
q2model = train_identity(model)


tensor([ 1.7637, -0.2675,  0.5906,  1.1975, -0.1213,  1.0661,  0.9292, -0.5465,
         0.8252])
Epoch [1/1000], Loss: 0.304771
Epoch [100/1000], Loss: 0.000735
Epoch [200/1000], Loss: 0.000238
Epoch [300/1000], Loss: 0.000253
Epoch [400/1000], Loss: 0.000119
Epoch [500/1000], Loss: 0.000069
Epoch [600/1000], Loss: 0.000108
Epoch [700/1000], Loss: 0.000075
Epoch [800/1000], Loss: 0.000076
Epoch [900/1000], Loss: 0.000053
Epoch [1000/1000], Loss: 0.000042


In [68]:

with torch.no_grad():
    out= model(torch.tensor([1.,23,2,1,4,5,6,7,3],))
print(out)

tensor([ 0.9779, 20.5914,  1.8880,  1.4047,  5.2232,  4.2357,  5.8981,  7.1204,
         2.5180])


In [69]:
with torch.no_grad():
    out= q2model(torch.tensor([1.,23,2,1,4,5,6,7,3],))
print(out)

tensor([ 0.4161,  0.3084,  0.9415,  0.5654, -0.0548, -0.5264, -0.6937, -0.3934,
        -0.1584])
